In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()
model=ChatOpenAI()


In [3]:
class BlogState(TypedDict):
    title: str
    content: str
    outline:str
    score:float

In [4]:
def create_outline(state:BlogState)->BlogState:
    #fetch title
    ttile=state['title']
    #call llm gen outline
    prompt=f'Generate adetailed outline for a blog on the topic {ttile}'
    outline=model.invoke(prompt).content

    #update state
    state['outline']=outline
    return state

In [5]:
def create_blog(state:BlogState)->BlogState:

    title=state['title']
    outline=state['outline']
    prompt=f'Write a detailed blog on the title -{title} using the following outline \n{outline}'
    content=model.invoke(prompt).content
    state['content']=content
    return state

In [6]:
def evaluate_score(state:BlogState)->BlogState:

    title=state['title']
    content=state['content']
    outline=state['outline']
    prompt = f"""
    You are an expert blog reviewer.

    Given the outline: "{outline}"

    Evaluate the blog titled "{title}" with the following content:
    "{content}"

    Give a single numeric score between 1 and 10 based on how well the blog matches the outline, is engaging, and well-written.

    Only return the numeric score.
    """
    score=model.invoke(prompt).content
    state['score']=score
    return state

In [7]:
graph=StateGraph(BlogState)

#nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('evaluate_score',evaluate_score)

#edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','evaluate_score')
graph.add_edge('evaluate_score',END)
workflow=graph.compile()

In [8]:
initial_state={'title':'Rise of AI in India'}

final_state=workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI in India', 'content': "\n\nI. Introduction\n\nA. Definition of AI\nArtificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, including learning, reasoning, and self-correction. AI technologies are designed to perform tasks that typically require human intelligence such as visual perception, speech recognition, decision-making, and language translation.\n\nB. Overview of the rise of AI in India\nIn recent years, India has emerged as a key player in the global AI landscape, with a growing number of startups, research initiatives, and government policies aimed at fostering the development and adoption of AI technologies in the country.\n\nC. Thesis statement\nThis blog will explore the historical background, current state, future trends, and ethical considerations of AI in India, highlighting the opportunities and challenges that come with the rise of AI in the country.\n\nII. Historical background of AI in India\n\nA. Early a

In [9]:
print(final_state['content'])




I. Introduction

A. Definition of AI
Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, including learning, reasoning, and self-correction. AI technologies are designed to perform tasks that typically require human intelligence such as visual perception, speech recognition, decision-making, and language translation.

B. Overview of the rise of AI in India
In recent years, India has emerged as a key player in the global AI landscape, with a growing number of startups, research initiatives, and government policies aimed at fostering the development and adoption of AI technologies in the country.

C. Thesis statement
This blog will explore the historical background, current state, future trends, and ethical considerations of AI in India, highlighting the opportunities and challenges that come with the rise of AI in the country.

II. Historical background of AI in India

A. Early adoption of AI technologies in India
India has a long history

In [10]:
print(final_state['outline'])

I. Introduction 
A. Definition of AI
B. Overview of the rise of AI in India 
C. Thesis statement 

II. Historical background of AI in India 
A. Early adoption of AI technologies in India 
B. Growth of AI startups in India 
C. Government initiatives to promote AI adoption 

III. Current state of AI in India 
A. AI applications in various industries such as healthcare, finance, and education 
B. Challenges faced by AI adoption in India 
C. Success stories of AI implementation in Indian companies 

IV. Future trends of AI in India 
A. Potential of AI to transform Indian industries 
B. Impact of AI on the job market in India 
C. Opportunities for further growth of AI in India 

V. Ethical considerations of AI in India 
A. Concerns about data privacy and security 
B. Ethics of AI algorithms and decision-making 
C. Regulatory framework for AI in India 

VI. Conclusion 
A. Recap of key points 
B. Implications of the rise of AI in India 
C. Call to action for businesses and policymakers to har

In [11]:
print(final_state['score'])

9
